# 对象引用、可变性和垃圾回收

## 引用式变量

在Python中，变量应当被理解为某一个特定内容的“标注”，而不是盛放这一内容的“容器”。即应当是先有“内容”，然后有变量；而不是先有变量然后有“内容”。

下述例子是对上述观点的佐证。下述例子首先定义了变量a，然后令b=a。若按照变量是“容器”的观点，上述b=a应当是将a中的数据拷贝到b中；若按照变量是“标注”的观点，上述b=a应当是将“标注”的身份赋予b，此时b和a是同一个内容的“标注”。

测试结果显示当改变变量a对应的list时，变量b对应的list也会同时改变。即变量a和变量b指向了同一个“内容”。虽然a、b的标识不同，但是其“内容”是共享的。

In [5]:
a = [1, 2, 3]
b = a
a.append(4)
print(b)
print(id(b), id(a))
b = 1
print(id(b), id(a))

[1, 2, 3, 4]
2110059546184 2110059546184
140718119729168 2110059546184


## 标识、相等性和别名

符号"=="以及关键字"is"均可用于判断两个变量是否相等。前者判断的是两个变量的值是否相等，后者则是判断两个变量的标识是否相同(id()函数返回的整数)。对于指向同一内存地址的不同变量，这些变量的值以及标识均相同；对于指向不同内存地址但是值相同的不同变量，这些变量的值相同，但是标识不同。

对于Python中的变量，每一个变量均有标识、类型和值。对象创建后，其标识在整个生命周期中不会改变。CPython中，id()函数返回对象的内存地址，其他Python解释器可能会返回其他含义的值。

### 元组的相对不可变性

名义上，元组是不可变类型，其中的元素不可改变。但是值得注意的是，对于存储在元组中的可变对象，元组中存储的实际上是可变对象的引用 —— 只要引用不发生改变，元组就认为这一元素没有改变，**元组并不会检查这一可变对象内的数据是否发生改变**。

将列表存入元组就是上述观点的一个例子。对于列表来说，列表是可变对象，其中的值可以进行删改而不影响其在内存中的地址；对于元组来说，元组存储的是列表的引用，只要引用不发生改变，对列表中元素的删增也不会报错。

下述例子创建了一个元组，其包含三个元素 —— 整数、字典和列表，并且利用变量a、b、c传入元组。依次修改上述三个变量后，观察结果。

* 对变量a的修改不会导致报错，并且这一修改不会影响元组中对应的值
* 对变量b的修改不会导致报错，并且这一修改会影响元组中对应的值
* 对变量c的修改不会导致报错，并且这一修改会影响元组中对应的值

上述结果实际上说明了几个问题：

1. 变量a验证了上述引用式变量的观点。整数是不可变数据，因此当对a进行赋值时，实际上是变量a变为整数“2”的标签。元组中仍保留的是整数“1”
2. 字典和列表是可变类型，因此当利用变量b或者c改变其元素时，其对应的数据会发生变化并且会反映到元组中对应的数据。

若直接对元组中的元素尝试使用索引获取并且进行赋值，不可变类型的元素尝试赋值会直接报错，可变类型的元素则可以使用各种原位的操作对其内容进行修改

In [4]:
a = 1
b = {"2": 2}
c = [3, 4, 5]
t1 = (a, b, c)
print(t1)

a = 2
b["2"] = 3
c.append(6)
print(t1)

t1[2].append(7)
t1[2][-1] = 6
t1[1]["3"] = 3
print(t1)
t1[0] = 0

(1, {'2': 2}, [3, 4, 5])
(1, {'2': 3}, [3, 4, 5, 6])
(1, {'2': 3, '3': 3}, [3, 4, 5, 6, 6])


TypeError: 'tuple' object does not support item assignment

## 浅拷贝与深拷贝

拷贝对象是常用操作，但是拷贝又是一个非常容易出错的操作。拷贝有如下几种可能：
* 拷贝前后两个变量指向同一个数据
* 浅拷贝：仅拷贝了最外层的容器。若该数据包含内部对象，则拷贝后的内部对象是原对象的引用
* 深拷贝：创建了所有元素的副本

### 浅拷贝

Python中的构造方法以及[:]是浅拷贝，这些操作仅拷贝了最外层的容器。若该数据包含内部对象，这些内部对象是共用的。若在拷贝后改变这些对象（赋值或是添加元素等），不可变类型的元素将会创建新对象。

下述例子准确描述了上述分析。对于通过构造方法复制得到的l2，若改变l2中的不可变类型元素则会创建一个新对象并且不会影响l1中对应位置的元素，若改变l2中可变类型元素则会同时改变l1中对应位置的元素。

In [18]:
# 浅拷贝
l1 = [3, [55, 44], (7, 8, 9)]

l2 = list(l1)
print("第一个元素的id()\n", id(l1[0]), id(l2[0]))
print("第二个元素的id()\n", id(l1[1]), id(l2[1]))
print("第三个元素的id()\n", id(l1[2]), id(l2[2]))

# 改变l2中的第一个元素
# 由于第一个元素是不可变类型
# 改变后l2[0]指向其他内存地址
# 此时l2[0]和l1[0]指向不同的内存地址
l2[0] = 0
print("\n改变l2中的第一个元素")
print("l1:", l1)
print("l2:", l2)
print("改变后第一个元素的id()\n", id(l1[0]), id(l2[0]))

# 改变l2中的第二个元素
# 由于第二个元素是可变类型
# 改变后l2[1]指向的内存地址不会改变
# 此时l2[1]和l1[1]指向相同的内存地址
l2[1].append(66)
print("\n改变l2中的第二个元素")
print("l1:", l1)
print("l2:", l2)
print("改变后第二个元素的id()\n", id(l1[1]), id(l2[1]))

# 改变l2中的第三个元素
# 由于第三个元素是不可变类型
# 改变后l2[2]指向的内存地址会改变
# 此时l2[2]和l1[2]指向不同的内存地址
l2[2] += (10, 11)
print(l1)
print("\n改变l2中的第三个元素")
print("l1:", l1)
print("l2:", l2)
print("改变后第三个元素的id()\n", id(l1[2]), id(l2[2]))

第一个元素的id()
 140717932886096 140717932886096
第二个元素的id()
 1726277473928 1726277473928
第三个元素的id()
 1726277414408 1726277414408

改变l2中的第一个元素
l1: [3, [55, 44], (7, 8, 9)]
l2: [0, [55, 44], (7, 8, 9)]
改变后第一个元素的id()
 140717932886096 140717932886000

改变l2中的第二个元素
l1: [3, [55, 44, 66], (7, 8, 9)]
l2: [0, [55, 44, 66], (7, 8, 9)]
改变后第二个元素的id()
 1726277473928 1726277473928
[3, [55, 44, 66], (7, 8, 9)]

改变l2中的第三个元素
l1: [3, [55, 44, 66], (7, 8, 9)]
l2: [0, [55, 44, 66], (7, 8, 9, 10, 11)]
改变后第三个元素的id()
 1726277414408 1726288276360


### 深拷贝

copy库提供的deepcopy()函数能够进行深拷贝，copy()函数则能够进行浅拷贝。

此外，numpy中可以使用.copy()直接执行深拷贝。值得注意的是，numpy使用copy()深拷贝后，若对其元素尝试使用id()函数获取内存地址会发现拷贝前后两个变量同一位置的元素会返回相同的id值，但是确实是完成了深拷贝（对一个变量对应的数据进行修改不会影响另一个变量对应的数据）

In [34]:
import numpy as np

l1 = np.zeros((2, 2), dtype=np.int32)
l2 = l1
l3 = l1.copy()

l3[0] = [1, 1]
print(l1)
print(l3)

print(id(l1), id(l2), id(l3))
print(id(l3[0]), id(l1[0]))

[[0 0]
 [0 0]]
[[1 1]
 [0 0]]
1726311337584 1726311337584 1726311226800
1726311252016 1726311252016


## 函数传参

Python使用的函数传参方式介于值传递和引用传递之间，本书使用的术语是：共享传递(call by sharing)，即Python传递的实际上是实参的引用。

对于传入的不可变对象，若在函数内对这一对象的形参进行了赋值操作，则仅有形参中的值会改变，而不会改变实参中的值。对于传入的可变类型对象，若在函数内对这一对象的形参进行了原位操作，则形参和实参中的值均会改变。

In [41]:
def f(a, b):
    print("运算前id")
    print("id(a): ", id(a))
    print("id(b): ", id(b))
    a += b
    print("运算后id")
    print("id(a): ", id(a))
    print("id(b): ", id(b))


# ------------ 不可变类型 ------------
print("不可变类型")
x_1 = 1
x_2 = 2
print("id(x_1): ", id(x_1))
print("id(x_2): ", id(x_2))
f(x_1, x_2)
print("x_1: ", x_1)
print("x_2: ", x_2)

# ------------ 可变类型 ------------
print("\n可变类型")
x_1 = [1, 2]
x_2 = [3, 4]
print("id(x_1): ", id(x_1))
print("id(x_2): ", id(x_2))
f(x_1, x_2)
print("x_1: ", x_1)
print("x_2: ", x_2)

不可变类型
id(x_1):  140717932886032
id(x_2):  140717932886064
运算前id
id(a):  140717932886032
id(b):  140717932886064
运算后id
id(a):  140717932886096
id(b):  140717932886064
x_1:  1
x_2:  2

可变类型
id(x_1):  1726277481224
id(x_2):  1726311505672
运算前id
id(a):  1726277481224
id(b):  1726311505672
运算后id
id(a):  1726277481224
id(b):  1726311505672
x_1:  [1, 2, 3, 4]
x_2:  [3, 4]


### 函数参数默认值

Python允许设定函数参数的默认值，此时应当避免使用可变对象作为参数的默认值。

若使用可变类型的对象作为参数默认值，当没有传入对应的实参并且对这一形参执行操作后，可能会导致默认值发生变化。这一变化也会反映到后续创建的实例中。

下述例子中，虽然仅对第二个实例进行了元素添加操作，但是第一个实例以及后续创建的第三个实例也受到了影响。

为了避免上述的问题，应当尽可能使用不可变类型作为参数的默认值，对于接收可变类型的形参可以使用None作为默认值。

In [44]:
class TestClass:

    def __init__(self, input_list=[]):
        self.input_list = input_list
    
    def _append(self, input_data):
        self.input_list.append(input_data)

test_class_1 = TestClass()
test_class_2 = TestClass()

# 向test_class_2的self.input_list添加一个元素
test_class_2._append(1)
print(test_class_1.input_list)

# 创建新实例
test_class_3 = TestClass()
print(test_class_3.input_list)

[1]
[1]
[]
